In [2]:
import psycopg2
import csv

def merge_and_export_to_csv():
    conn_params = {
        "dbname": "admin",
        "user": "postgres",
        "password": "admin",
        "host": "localhost",
        "port": 5433
    }

    try:
        with psycopg2.connect(**conn_params) as conn:
            with conn.cursor() as cursor:

                cursor.execute("SELECT COUNT(*) FROM Clients_list")
                num_rows = cursor.fetchone()[0]
                

                if num_rows > 100:
                    fetch_limit = 50
                elif num_rows > 50:
                    fetch_limit = 20
                else:
                    fetch_limit = num_rows


                cursor.execute("""
                    SELECT c.ClientID, c.Name, c.fullAddress, c.Postcode, c.mobileInfo,
                           a.AreaType, cf.CollectionFrequency, cs.LastCollectionDate, cs.NextCollectionDate,
                           wc.WasteType, wc.UnitPricePerKg, wd.DisposalDate, wd.QuantityInKg,
                           br.BillingDate, br.TotalAmount, bd.QuantityInKg as BillingQuantity, bd.SubTotal
                    FROM Clients_list c
                    JOIN AreaTypes a ON c.AreaTypeID = a.AreaTypeID
                    JOIN CollectionSchedule cs ON c.ClientID = cs.ClientID
                    JOIN CollectionFrequencies cf ON cs.FrequencyID = cf.FrequencyID
                    JOIN WasteCategories wc ON cs.WasteCategoryID = wc.WasteCategoryID
                    LEFT JOIN WasteDisposal wd ON c.ClientID = wd.ClientID AND wc.WasteCategoryID = wd.WasteCategoryID AND cs.FrequencyID = wd.FrequencyID
                    LEFT JOIN BillingRecords br ON c.ClientID = br.ClientID
                    LEFT JOIN BillingDetails bd ON br.BillingID = bd.BillingID
                    LIMIT %s
                """, (fetch_limit,))


                with open('merged_postgres.csv', mode='w', newline='') as file:
                    writer = csv.writer(file)

                    writer.writerow([
                        "ClientID", "Name", "FullAddress", "Postcode", "MobileInfo",
                        "AreaType", "CollectionFrequency", "LastCollectionDate", "NextCollectionDate",
                        "WasteType", "UnitPricePerKg", "DisposalDate", "QuantityInKg",
                        "BillingDate", "TotalAmount", "BillingQuantity", "SubTotal"
                    ])
                    

                    for row in cursor.fetchall():
                        writer.writerow(row)

                print("CSV file 'merged_postgres.csv' has been created successfully.")

    except psycopg2.Error as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    merge_and_export_to_csv()

CSV file 'merged_postgres.csv' has been created successfully.
